<img src="./images/logo.png" alt="Drawing" style="width: 500px;"/>

<div class="alert alert-block alert-danger">
<b>Important:</b> This exercise requires the completion of Exercise 3: Exploring Retail Data with Apache Spark.</div>

# **Exercise 4:** Creating an AI Agent to analyze the data.

In this exercise, you'll explore how to harness the power of HPE Private Cloud AI’s **NVIDIA Inference Microservices (NIM)**, featuring Meta's **Llama 3.1 8b Instruct**, to create your very own **AI-powered Data Analyst Agent**. This agent will interact with your prepared data and help you analyze, summarize, and derive insights—all with natural language.

HPE PCAI provides scalable, containerized access to state-of-the-art models like Llama 3.1, enabling low-latency, high-throughput inferencing—perfect for building intelligent agents that can reason over structured and unstructured data.

Your journey in this exercise will include:
- Integrating your previously prepared datasets with the inference workflow.
- Configuring your AI agent so that it leverages Llama 3.1 8b via NVIDIA Inference Microservices.
- Crafting prompts and building logic for your AI agent to act like a data analyst.
- Interacting with your AI agent using natural language within a Jupyter notebook.

By the end of this exercise, you’ll be able to prototype a lightweight, intelligent AI assistant that can query, explain, and generate insights—turning raw data into valuable knowledge with just a few prompts.

Let’s get started and build your first Data Analyst AI Agent!

## **1. Agent Configuration**

This section covers the configuration of the agent, including:  
* Defining the data context that the agent will interact with  
* Setting up the routine the agent will follow as a system prompt (embedding the data context)  
* Establishing the list of tools available for the agent to complete its tasks  

<div class="alert alert-block alert-danger">
    <b>Important:</b> Set your <b>Username</b>, your <b>Domain</b> and the name of your <b>Presto connection</b> (catalog) here !
</div>

In [4]:
USERNAME="vince"
DOMAIN="hpepcai.ezmeral.demo.local"
CATALOG="deltavince"
SCHEMA="default"

In [5]:
# 0. Import Librairies
import os
from pathlib import Path
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA
from llama_index.embeddings.nvidia import NVIDIAEmbedding
import json
import inspect
from pandas import DataFrame
import psycopg2

We start by defining a function to retrieve and refresh the NVIDIA JWT authentication token from a secure file path, as the token expires every 30 minutes and must be updated regularly to maintain API access.

In [8]:
# 1. Read JWT Token
def get_nvidia_auth_token():
    %update_token
    token_path = Path("/etc/secrets/ezua/.auth_token")
    if token_path.exists():
        with open(token_path, "r") as f:
            return f.read().strip()
    raise ValueError("NVIDIA auth token not found at /etc/secrets/ezua/.auth_token")

nvidia_api_key = get_nvidia_auth_token()

Token successfully refreshed.


We start by defining a function to retrieve and refresh the NVIDIA JWT authentication token from a secure file path, as the token expires every 30 minutes and must be updated regularly to maintain API access.

In [10]:
# 3. NVIDIA NIM Setup
llm = NVIDIA(
    base_url="https://llama-3-1-8b-0b33052f-predictor-ezai-services.hpepcai.ezmeral.demo.local/v1",
    model="meta/llama-3.1-8b-instruct",
    api_key=nvidia_api_key,
    temperature=0.1,
    max_tokens=1024
)
Settings.llm = llm

Retrying request to /models in 0.492529 seconds
Retrying request to /models in 0.945308 seconds
Retrying request to /models in 1.819459 seconds


APIConnectionError: Connection error.

In [12]:
from pyhive import presto
from pandas import DataFrame
import json

In [13]:
# 4. Presto Connection
def get_presto_connection():
    return presto.connect(
        host=f"ezpresto.{DOMAIN}",
        port=443,
        catalog=CATALOG,
        schema=SCHEMA,
        protocol='https'
    )

In [14]:
# 5. Delta Table Schema Query (fixed connection handling)
def query_delta_dictionary():
    query = f'''
    SELECT 
        table_schema as "DatabaseName",
        table_name as "TableName", 
        column_name as "ColumnName",
        data_type as "ColumnType"
    FROM {CATALOG}.information_schema.columns
    WHERE table_schema NOT IN ('information_schema', 'sys')
      AND table_schema = '{SCHEMA}'
    '''
    conn = None
    try:
        conn = get_presto_connection()
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        table_dictionary = DataFrame(results, 
                                   columns=["DatabaseName", "TableName", "ColumnName", "ColumnType"])
        return json.dumps(table_dictionary.to_json())
    finally:
        if conn:
            conn.close()



In [16]:
# 6. System Prompt Setup
db_dictionary = query_delta_dictionary()

system_prompt = f"""
You are an advanced data analyst for a retailer company, specializing in analyzing data from our Delta Lake tables accessed via Presto. Your primary responsibility is to assist users by answering business-related questions using SQL queries. Follow these steps:

1. Understanding User Requests
   - Users provide business questions in plain English.
   - Extract relevant data points needed to construct a meaningful response.

2. Generating SQL Queries
   - Construct an optimized Presto SQL query to retrieve the necessary data from Delta tables.
   - The query must be a **single-line string** without carriage returns or line breaks.
   - Ensure the query uses proper catalog.schema.table references (format: {CATALOG}.{SCHEMA}.table_name)
   - The metadata of available tables and columns is in this json structure: 
     {db_dictionary}
   - Apply appropriate filtering, grouping, and ordering to enhance performance.
   - Presto-specific considerations:
     * Use `DATE()` for date casting instead of `::date`
     * String concatenation uses `||` not `+`
     * For approximate counts, consider `approx_distinct()` 
   - Don't display the SQL queries unless specifically asked

3. Executing the Query
   - Run the SQL query on our Presto system and retrieve the results efficiently.

4. Responding to the User
   - Convert the query results into a **concise, insightful, and plain-English response**.
   - Present the information in a clear, structured, and user-friendly manner.
   - For large results, consider summarizing trends instead of listing all data points.

You have access to these tools:
- `query_delta_database`: For executing Presto SQL queries on Delta tables
- `query_delta_dictionary`: For fetching metadata about tables and columns

Always use `query_delta_database` when the user asks for data stored in our Delta tables.
Important: Never suggest queries that would modify data - we only allow read operations.
"""

TypeError: 'Connection' object does not support the context manager protocol

In [15]:
# 7. Query Delta Tables
def query_delta_database(sql_statement):
    try:
        # Ensure the query uses the proper catalog.schema.table format
        query_statement = sql_statement.strip().replace('\n', ' ')
        
        # If it's a simple table query without catalog/schema, add them
        if 'FROM ' in query_statement and '.' not in query_statement.split('FROM ')[1].split()[0]:
            table_ref = query_statement.split('FROM ')[1].split()[0]
            query_statement = query_statement.replace(
                f'FROM {table_ref}', 
                f'FROM {CATALOG}.{SCHEMA}.{table_ref}'
            )
        
        with get_presto_connection() as conn:
            cursor = conn.cursor()
            cursor.execute(query_statement)
            
            if cursor.description:
                columns = [desc[0] for desc in cursor.description]
                data = cursor.fetchall()
                df = DataFrame(data, columns=columns)
                return json.dumps(df.to_dict(orient='records'))
            else:
                return json.dumps({"message": "Query executed successfully"})
    except Exception as e:
        return json.dumps({"error": str(e)})

In [ ]:
# 8. Agent Conversation Function
def run_agent_conversation(user_query):
    from llama_index.core.llms import ChatMessage
    
    messages = [
        ChatMessage(role="system", content=system_prompt),
        ChatMessage(role="user", content=user_query)
    ]
    
    response = llm.chat(messages)
    return str(response)  # Changed from response.content to str(response)

In [ ]:
# 9. Example Usage (with proper string termination)
response = run_agent_conversation("What are the top 5 selling products by revenue?")
print(response)

## Agent Runtime
This section covers the code executed while the agent is in action, including:
* Preparing the tools for use by the agent
* The agent's runtime function

In [ ]:
from typing import Dict, Any, Callable
import inspect
import json

def function_to_schema(func: Callable) -> Dict[str, Any]:
    """Convert a Python function to a tool schema compatible with NVIDIA LLM
    
    Args:
        func: The Python function to convert
        
    Returns:
        Dictionary containing the function schema in NVIDIA-compatible format
    """
    sig = inspect.signature(func)
    docstring = inspect.getdoc(func) or ""
    
    # Extract parameter information
    parameters = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    for name, param in sig.parameters.items():
        if name == "self":
            continue
            
        param_type = "string"  # default type
        if param.annotation != inspect.Parameter.empty:
            if param.annotation == str:
                param_type = "string"
            elif param.annotation == int:
                param_type = "integer"
            elif param.annotation == float:
                param_type = "number"
            elif param.annotation == bool:
                param_type = "boolean"
        
        parameters["properties"][name] = {
            "type": param_type,
            "description": ""  # Can be enhanced with parameter-specific docs
        }
        
        if param.default == inspect.Parameter.empty:
            parameters["required"].append(name)
    
    return {
        "name": func.__name__,
        "description": docstring,
        "parameters": parameters
    }

In [ ]:
# 10. Prepare Tools for Agent
from llama_index.core.llms import ChatMessage
from typing import List, Dict, Any

tools = [query_postgres_database]
tool_schemas = [function_to_schema(tool) for tool in tools]
tools_map = {tool.__name__: tool for tool in tools}

def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)

def convert_to_chat_message(message: Dict[str, Any]) -> ChatMessage:
    """Convert dictionary message to LlamaIndex ChatMessage"""
    return ChatMessage(
        role=message["role"],
        content=message["content"],
        additional_kwargs=message.get("additional_kwargs", {})
    )

from IPython.display import display, Markdown
import time

def run_full_turn(system_message: str, messages: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    chat_messages = [convert_to_chat_message(msg) for msg in messages]
    
    while True:
        # Initialize streaming
        full_response = []
        response_buffer = ""
        out = display(Markdown(""), display_id=True)
        
        # Get streaming response with token awareness
        response_stream = llm.stream_chat(
            chat_messages,
            max_tokens=4096,  # Adjust based on your model's limits
            temperature=0.1
        )
        
        # Process stream with token-aware chunking
        for chunk in response_stream:
            content = chunk.delta
            if content:
                response_buffer += content
                full_response.append(content)
                
                # Display when we hit natural breaks or every 20 tokens
                if len(response_buffer.split()) >= 20 or content.endswith(('\n', '.', '!', '?')):
                    out.update(Markdown("".join(full_response)))
                    response_buffer = ""
                    time.sleep(0.05)  # Natural reading speed
        
        # Final update to ensure complete display
        out.update(Markdown("".join(full_response)))
        
        # Store complete response
        response_dict = {
            "role": "assistant",
            "content": "".join(full_response),
            "additional_kwargs": getattr(response_stream, "additional_kwargs", {})
        }
        messages.append(response_dict)
        
        # Handle tool calls (unchanged)
        additional_kwargs = response_dict.get("additional_kwargs", {})
        if "tool_calls" in additional_kwargs:
            for tool_call in additional_kwargs["tool_calls"]:
                result = execute_tool_call(tool_call, tools_map)
                result_message = {
                    "role": "tool",
                    "content": result,
                    "tool_call_id": tool_call.get("id", ""),
                    "name": tool_call["function"]["name"]
                }
                messages.append(result_message)
                chat_messages.append(convert_to_chat_message(result_message))
        else:
            break
    
    return messages

## Running the Agent
### Sample Questions:
1. What are our top-selling products by revenue and quantity sold?
2. Who are our top 10 customers by total spend and order frequency?
3. Which products have the lowest stock levels relative to their sales velocity?
4. Which product categories generate the highest profit margins?
5. What is our order fulfillment rate and average time to fulfill orders?
6. How has our customer base grown over time?
7. What are the seasonal trends in our product categories?
8. What products are frequently purchased together?
9. What percentage of customers make repeat purchases?
10. Which customer segments are most profitable when considering acquisition cost and lifetime value?

In [ ]:
# Updated imports
from typing import AsyncIterator, Iterator
import sys
import time

# 10. Prepare Tools for Agent (unchanged)
tools = [query_postgres_database]
tool_schemas = [function_to_schema(tool) for tool in tools]
tools_map = {tool.__name__: tool for tool in tools}

# Modified agent interaction with streaming
def run_agent_interaction():
    messages = [{"role": "system", "content": system_prompt}]
    
    while True:
        user_input = input("\nUser (type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
            
        messages.append({"role": "user", "content": user_input})
        messages = run_full_turn(system_prompt, messages)
        
        # Display any tool results
        for msg in reversed(messages):
            if msg.get("role") == "tool" and "content" in msg:
                print(f"\n[Database Result]: {msg['content']}")
                break

if __name__ == "__main__":
    run_agent_interaction()

## **1. Connecting a Data Source in Unified Analytics**

**HPE Ezmeral Unified Analytics** allows users to connect multiple types of internal and external data sources - from SQL servers to Snowflake, Terradata and Oracle databases - and make the files, objects and tables within them available to any tool or application running on Unified Analytics.

In this section, you will learn how to make a data connection using the Delta Tables you created in Exercise 1. 

### Connect Delta Tables as Data Source using Apache Hive.

Let's take those Delta Tables you created in Exercise 1 and make them available to other applications in Unified Analytics by connecting them as an **Data Source**. Apache Hive gives an SQL-like interface to query data stored in various databases and file systems, like the delta tables you created in Exercise 1. This will allow you to use EzPresto to turn them into datasets later in the exercise. 

1. Navigate back to the Unified Analytics dashboard.
1. In the sidebar navigation menu, select `Data Engineering` > `Data Sources`.
1. Under the `Structured Data` tab, click `Add New Data Source`.
1. Under **Hive**, click `Create Connection`.

- **Name**: `retail`
- **Hive Metastore:** `discovery`
- **Data Dir:** `file:/data/shared/retail-data/delta-tables`
- **File Type:** `PARQUET`
5. Click `Connect`.



<img src="./images/exercise2/connect-dl.png" alt="Drawing" style="width: 25%;"/>

6. Under the `Structured Data` tab, you will now see your connected data source. 

### Viewing and Querying Data from Data Sources

Now that our Delta Tables are available via a Data Source, we can leverage the native data tools within Unified Analytics to run queries and create datasets.

1. Click on the three dots in the top right hand corner of the newly created data source.
1. Click `Change to public access`. In the dialog box that appears, click `Proceed`.
1. Next, select `Query using Data Catalog`.

<img src="./images/exercise2/title.png" alt="Drawing" style="width: 35%;"/>

4. Under `Connected Data Sources`, look for the `retail` group.
5. Under the `retail` group, check the `default` box. 
6. Select the datasets for all three countries. 


<img src="./images/exercise2/datacatalog.png" alt="Drawing" style="width: 70%;"/>

7. Click `Selected Datasets` in the top left corner.
8. Click `Query Editor`.

Here, you are introduced to the the **Unified Analytics Query Editor** where you can directly query data sources from specific datasets and data tables - all from within the Unified Analytics user interface! 

<img src="./images/exercise2/QueryEditor.png" alt="Drawing" style="width: 75%;"/>

9. Next, we're going to run an SQL query which will combine the data from our three tables (czech, germany, and swiss) in the retail.default schema. This will merge all columns from the czech and germany tables, and select specific columns from the swiss table whilst also applying a transformation to the country column. We'll also limit our final result set to 1000 rows. And all in a nice UI!

    Paste the following SQL Query into the `SQL Query` field and 
    click `Run`.

```sql
SELECT * FROM retail.default.czech UNION ALL SELECT * FROM retail.default.germany UNION ALL ( SELECT PRODUCTID , PRODUCT , TYPE , UNITPRICE , UNIT , QTY , TOTALSALES , CURRENCY , STORE , (CASE WHEN (country = 'Swiss') THEN 'Switzerland' ELSE country END) COUNTRY , YEAR FROM retail.default.swiss ) LIMIT 1000

```

10. Expand the resulting query to visually validate it.
11. Under `Actions` (top-left corner of the table), click `Save as View`.

<img src="./images/exercise2/query-results.png" alt="Drawing" style="width: 75%;"/>


12. Name the View `retail`. 
13. We'll want to save the schema of this new table as a Custom Schema. Under `Schema`, select `+ Add new schema` and name it `retailschema`.

<img src="./images/exercise2/save-as-view.png" alt="Drawing" style="width: 45%;"/>

You have now saved the dataset resulting from your SQL query as an **Asset**. Assets are made available to other applications via EzPresto, as you will explore in Exercise 3. 

## **2. Cached Assets in HPE Ezmeral Unified Analytics**

To view our saved Assets, including the one that we just created:

1. In Unified Analytics, under the sidebar navigation menu, select `Data Engineering` > `Cached Assets`. 
1. You should see an asset with Name `retail`. 
1. Click the three dots associated with the `retail` asset.

<img src="./images/exercise2/cachedasset.png" alt="Drawing" style="width: 75%;"/>

4. Click `View Columns`. 
5. Validate there are eleven fields. 

Should you ever wish to run futher queries on a Cached Asset in the future, check the box next to an asset and click `Query Editor`. 

## **3. Jupyter Magic Commands on HPE Ezmeral Unified Analytics**

Jupyter Notebooks Magic functions, also known as magic commands or magics, are commands that you can execute within a code cell.   
Magics are not code of any language, but are shortcuts that extends the capabilities of a notebook. 

There are two types of magic commands - **Line** and **Cell** magic commands:

**Line magic** commands do not require a cell body and start with a single % character.  
**Cell magic** commands start with %% and require additional lines of input (a cell body). 

### HPE Ezmeral Magic Commands 
**HPE Ezmeral Unified Analytics Software** supports both Line and Cell magic commands and includes custom commands that allow for users to interact with other tools native to Unified Analytics directly within notebooks.

We can check out the full list of custom HPE magic commands by running `%command`.

In [ ]:
# The %commands command lists the magic commands and SDKs that are customized by Hewlett Packard Enterprise and are available in this notebook.
%commands

### Updating the Cached JWT Token

In a Jupyter notebook, a JWT token (JSON Web Token) is a compact and URL-safe means of representing authentication information to be transferred between other notebook servers or external applications. It is commonly used for securely authenticating and authorizing users within Jupyter environments, allowing them to access resources and execute code while ensuring their identity and permissions are properly validated.

When working in Jupyter notebooks for long durations, particularly when making calls to other applications, the JWT token can expire and result in an error when attempting to make calls. This is particularly relevant for working on a Jupyter notebook within **HPE Ezmeral Unified Analytics**, which provides users to leverage a plethora of external tools within the notebook (Such as Spark, Livy and Presto).

**If you encounter a JWT token expiration error while running cells in a Jupyter notebook**, you can resolve it by running the `%update_token` magic command.  
This function updates the JWT in environment variables and any other locations where the token is utilized. 
  
Ideally, it is good practice to refresh the token prior to making external connections. Some examples relevant to the Smart Retail Experience demo include:

- Authentication when establishing a connection with PrestoDB.
- Authentication with local s3 minio object storage.  
- Authentication with KServe external API.  

In [ ]:
%update_token

### Directly interacting with connected SQL databases using the SQL Magic Command

Using the `%sql` magic command, you can directly query SQL databases from Data Sources you have connected to **HPE Ezmeral Unified Analytics Software** from within Jupyter notebook cells! When you run the notebook cell containing `%sql` and your SQL query, the magic command sends the query to the database, runs the query, and retrieves the result.

This is made possible by the native integration of EzPresto into Unified Analytics. **However, the Data Source must be made publicly available.**

To change the access of a Data Source from `private` to `public`:

1. Navigate back to the Unified Analytics dashboard.
1. In the sidebar navigation menu, select `Data Engineering` > `Data Sources`.
1. Under the three dots in the top corner of the Data Source of interest, click `Change to public access`.

<div class="alert alert-block alert-danger">
<b>Important:</b> Wait until a confirmation message appears stating that the source is publicly available.
</div>

Now, let's try the `%sql` magic command to interact with our Delta Tables. 

In [ ]:
%sql select * from retail.retail.czech limit 10

We can also save the output of our command as a Python variable!

In [ ]:
result = %sql select * from retail.retail.czech limit 10
print(result)

# **Conclusion**

In this exercise, you learned how **EzPresto** on **HPE Ezmeral Unified Analytics** makes connecting internal and external data sources to your applications, such this notebook hosted on Unified Analytics, a snap. You learned how to leverage the data engineering tools available within the Unified Analytics interface, including the Query Editor, to create datasets from your data sources that could then be shared and quered using HPE Magic Commands inside Unified Analytics-hosted Jupyter Notebooks.

In the next exercise, you will learn how to use visualize the datasets you have created using **Apache Superset**!